In [ ]:
!pip install -Uqq fastai duckduckgo_search

# Download Images from Duck Duck Go Search Engine

In [ ]:
from duckduckgo_search import DDGS
from fastcore.all import *

def search_images(query, max_images=30):
    """
    This function returns a list of image urls from duck duck go search
    for a particular term.
    """
    print(f"Searching for {query}")
    ddgs = DDGS()
    ddgs_images_gen = ddgs.images(query, max_results=max_images)
    return L(ddgs_images_gen).itemgot('image')

In [ ]:
urls = search_images("rusted roof", max_images=1)
urls[0]

Download a sample image and display it.

In [ ]:
from fastdownload import download_url
dest = 'rusted_roof.jpg'
download_url(urls[0], dest)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256, 256)

In [ ]:
download_url(search_images('pristine_roof', max_images=1)[0], 'pristine_roof.jpg')
Image.open('pristine_roof.jpg').to_thumb(256, 256)

Download images & resize.

In [ ]:
from time import sleep
queries = 'rusted roof', 'pristine roof'
path = Path('roofs')

for query in queries:
    dest = (path / query.split()[0])
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(query))
    sleep(10)
    resize_images(path/ query.split()[0], max_size=400, dest=path/ query.split()[0])

# Finetune the pretrained model ResNet

Remove any photos that might be corrupted.

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
print(f"Number of files deleted: {len(failed)}")

Create a `DataBlock` object containing the training & validation set.

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label, 
    item_tfms=[Resize(192, method='flip')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=7)

**Using ResNet 18 to train the model**

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(5)

The model achieves an accuracy of 91% on the validation set.

# Seeing the Prediction

In [ ]:
print("Sample test photo")
Image.open('rusted_roof.jpg').to_thumb(256, 256)

In [ ]:
is_rusted, _, probs  = learn.predict(PILImage.create('rusted_roof.jpg'))
print(f"This roof is: {is_rusted}")
print(f"Probability it's a {is_rusted} roof is: {probs[1]}")

In [ ]:
print("Sample test photo")
Image.open('pristine_roof.jpg').to_thumb(256, 256)

In [ ]:
is_rusted, _, probs  = learn.predict(PILImage.create('pristine_roof.jpg'))
print(f"This roof is: {is_rusted}")
print(f"Probability it's a {is_rusted} roof is: {probs[0]}")